# Basic Configuration

In [67]:
# ========================== edit here ========================
START_TIME = 8
END_TIME = 21
UNIT = 1    # 1 hour
#UNIT = 0.5 # 30 mins
DAYS = ["월", "화", "수", "목", "금"]

available = {
    "김민규": {
        "월": [(15, 18)],
        "화": [(15, 18)],
        "수": [(15, 18)],
        "목": [(15, 18)],
        "금": [(15, 18)],
    },
    "김지효": {
        "화": [(11, 16)],
        "수": [(11, 16)],
    },
    "Justin": {
        "화": [(16, 19)],
        "수": [(16, 17)],
        "목": [(16, 17)],
        "금": [(13, 15)],
    },
    "김현경": {
        "화": [(8, 9)],
        "수": [(8, 10)],
    },
    "윤찬웅": {
        "월": [(8, 10)],
        "화": [(9, 10)],
        "목": [(8, 10)],
        "금": [(8, 10)],
    },
    "이동훈": {
        "월": [(14, 18)],
        "화": [(14, 18)],
        "목": [(14, 18)],
        "금": [(9, 12), (15, 21)],
    },
    "제영선": {
        "월": [(9, 12), (14, 17)],
        "화": [(15, 17)],
        "수": [(10, 17)],
        "목": [(15, 17)],
        "금": [(9, 12)],
    },
    "조민우": {
        "월": [(19, 21)],
        "화": [(19, 21)],
        "수": [(15, 21)],
    },
    "조유진": {
        "월": [(10, 12), (16, 21)],
        "화": [(19, 21)],
        "수": [(16, 21)],
        "목": [(19, 21)],
        "금": [(10, 21)],
    },
    "신수용": {
        "월": [(10, 15)],
        "화": [(11, 14), (16, 21)],
        "수": [(10, 15)],
        "목": [(11, 14), (16, 21)],
        "금": [(9, 18)],
    },
    "이성준": {
        "월": [(12, 21)],
        "화": [(9, 12), (16.5, 21)],
        "수": [(12, 21)],
        "목": [(9, 12), (16.5, 21)],
        "금": [(9, 21)],
    },
    "백준규": {
        "월": [(9, 12)],
        "화": [(9, 12)],
        "수": [(9, 12)],
        "목": [(9, 12)],
    },
}
# ==============================================================



In [68]:
slot_start_times = [
    START_TIME + UNIT*i
    for i in range(int((END_TIME - START_TIME)/UNIT))
]
slots = {
    day: {
        st: []
        for st in slot_start_times
    } for day in DAYS
}

for person in available:
    for day in available[person]:
        for start_t, end_t in available[person][day]:
            for st in slot_start_times:
                if st >= start_t and st+UNIT <= end_t:
                    slots[day][st].append(person)

time_ratio = {}
for person in available:
    amount = 0
    for day in available[person]:
        for st, et in available[person][day]:
            amount += et - st
    time_ratio[person] = amount
total = sum(time_ratio.values())
for person in time_ratio:
    time_ratio[person] /= total
time_ratio

{'김민규': 0.07075471698113207,
 '김지효': 0.04716981132075472,
 'Justin': 0.0330188679245283,
 '김현경': 0.014150943396226415,
 '윤찬웅': 0.0330188679245283,
 '이동훈': 0.09905660377358491,
 '제영선': 0.09433962264150944,
 '조민우': 0.04716981132075472,
 '조유진': 0.12735849056603774,
 '신수용': 0.1650943396226415,
 '이성준': 0.21226415094339623,
 '백준규': 0.05660377358490566}

In [69]:
import random
import copy

def get_random_assignment(slots):
    assignments = {
        day: {
            st: None
            for st in slot_start_times
        } for day in DAYS
    }
    for day in slots:
        for st in slots[day]:
            if slots[day][st]:
                assignments[day][st] = random.choice(slots[day][st])
    return assignments

def mutate_assignment(assignments, slots):
    mutant = copy.deepcopy(assignments)
    for day in slots:
        for st in slots[day]:
            if slots[day][st]:
                if random.random() < 0.1:
                    # mutate 10%
                    mutant[day][st] = random.choice(slots[day][st])
    return mutant

def num_fragment(seq):
    count = 1
    for i in range(1, len(seq)):
        if seq[i] != seq[i-1]:
            count +=1
    return count


def num_short_or_long_fragment(seq):
    len_fragments = [1]
    for i in range(1, len(seq)):
        if seq[i] != seq[i-1]:
            len_fragments.append(1)
        else:
            len_fragments[-1] += 1
    # 1시간 혹은 5시간 이상 슬랏에 패널티
    return sum([not (2 <= l <= 4) for l in len_fragments])

def compute_fitness(assignments, ideal_time_ratio, alpha=0.5, verbose=False):
    fragment_fitness = 0
    ratio_fitness = 0
    for day in assignments:
        assignees = [assignments[day][st] for st in slot_start_times]
        fragment_fitness += num_fragment(assignees)/len(slot_start_times)
        fragment_fitness += num_short_or_long_fragment(assignees)/num_fragment(assignees) # penalty
    fragment_fitness /= 2

    current_time_ratio = {person: 0 for person in ideal_time_ratio}
    for day in assignments:
        for person in assignments[day].values():
            if person is not None:
                current_time_ratio[person] += 1

    total = sum(current_time_ratio.values())
    for person in current_time_ratio:
        current_time_ratio[person] /= total

    if verbose:
        print(f"name\tideal\tactual")
    for person in current_time_ratio:
        ratio_fitness += abs(ideal_time_ratio[person]
            - current_time_ratio[person])
        if verbose:
            print(f"{person}\t{ideal_time_ratio[person]:.3f}\t{current_time_ratio[person]:.3f}")

    return alpha * fragment_fitness/len(DAYS) + (1 - alpha) * ratio_fitness

# Run Genetic Algorithm

In [70]:
NUM_POPULATIONS = 50
NUM_GENERATIONS = 100
NUM_ELITES = 10
ALPHA = 0.7 # [0, 1] 0에 가까울 수록 분배 우선, 1에 가까울 수록 연속적 배정 우선

# initial population
populations = [
    get_random_assignment(slots)
    for i in range(NUM_POPULATIONS)
]

def get_best_n(populations, n, alpha):
    fitnesses = []
    for individual in populations:
        fitnesses.append((
            compute_fitness(individual, time_ratio, alpha),
            individual
        ))
    fitnesses = sorted(fitnesses, key=lambda t: t[0])
    return [i for f, i in fitnesses[:n]]

print("Scheduling...", end=" ")
for gen in range(NUM_GENERATIONS):
    elites = get_best_n(populations, NUM_ELITES, alpha=ALPHA)
    new_populations = []
    for i in range(NUM_POPULATIONS - NUM_ELITES):
        new_populations.append(mutate_assignment(random.choice(elites), slots))
    populations = elites + new_populations
    assert len(populations) == NUM_POPULATIONS
print("Done")

Scheduling... Done


In [71]:
import pandas as pd

N = 3
best_individuals = get_best_n(populations, N, alpha=ALPHA)

for i in range(N):
    assignments = best_individuals[i]
    rows = []
    print(f"=================== Option #{i+1} ===================")
    compute_fitness(assignments, time_ratio, verbose=True)
    for day in assignments:
        for st in assignments[day]:
            rows.append([day, st, assignments[day][st]])
    df = pd.DataFrame(data=rows,
        columns=["day", "start_time", "assignee"]
        ).pivot("start_time", "day", "assignee")
    display(
        df[DAYS]
    )

=================== Option #1 ===================
name	ideal	actual
김민규	0.071	0.077
김지효	0.047	0.046
Justin	0.033	0.000
김현경	0.014	0.046
윤찬웅	0.033	0.092
이동훈	0.099	0.108
제영선	0.094	0.046
조민우	0.047	0.031
조유진	0.127	0.108
신수용	0.165	0.169
이성준	0.212	0.215
백준규	0.057	0.062


day,월,화,수,목,금
start_time,,,,,
8,윤찬웅,김현경,김현경,윤찬웅,윤찬웅
9,윤찬웅,이성준,김현경,윤찬웅,윤찬웅
10,백준규,이성준,제영선,백준규,이동훈
11,백준규,신수용,제영선,백준규,이동훈
12,신수용,신수용,신수용,신수용,조유진
13,신수용,김지효,신수용,신수용,조유진
14,이동훈,김지효,신수용,이동훈,조유진
15,이동훈,김지효,제영선,이동훈,이성준
16,이동훈,김민규,김민규,김민규,이성준


=================== Option #2 ===================
name	ideal	actual
김민규	0.071	0.077
김지효	0.047	0.046
Justin	0.033	0.000
김현경	0.014	0.046
윤찬웅	0.033	0.092
이동훈	0.099	0.108
제영선	0.094	0.046
조민우	0.047	0.031
조유진	0.127	0.108
신수용	0.165	0.169
이성준	0.212	0.215
백준규	0.057	0.062


day,월,화,수,목,금
start_time,,,,,
8,윤찬웅,김현경,김현경,윤찬웅,윤찬웅
9,윤찬웅,이성준,김현경,윤찬웅,윤찬웅
10,백준규,이성준,제영선,백준규,이동훈
11,백준규,신수용,제영선,백준규,이동훈
12,신수용,신수용,신수용,신수용,조유진
13,신수용,김지효,신수용,신수용,조유진
14,이동훈,김지효,신수용,이동훈,조유진
15,이동훈,김지효,제영선,이동훈,이성준
16,이동훈,김민규,김민규,김민규,이성준


=================== Option #3 ===================
name	ideal	actual
김민규	0.071	0.077
김지효	0.047	0.046
Justin	0.033	0.000
김현경	0.014	0.046
윤찬웅	0.033	0.092
이동훈	0.099	0.108
제영선	0.094	0.046
조민우	0.047	0.031
조유진	0.127	0.108
신수용	0.165	0.169
이성준	0.212	0.215
백준규	0.057	0.062


day,월,화,수,목,금
start_time,,,,,
8,윤찬웅,김현경,김현경,윤찬웅,윤찬웅
9,윤찬웅,이성준,김현경,윤찬웅,윤찬웅
10,백준규,이성준,제영선,백준규,이동훈
11,백준규,신수용,제영선,백준규,이동훈
12,신수용,신수용,신수용,신수용,조유진
13,신수용,김지효,신수용,신수용,조유진
14,이동훈,김지효,신수용,이동훈,조유진
15,이동훈,김지효,제영선,이동훈,이성준
16,이동훈,김민규,김민규,김민규,이성준
